In [45]:
from skimage.transform import resize
import numpy as np
from skimage.util import random_noise
from PIL import Image

def gen_noise_mask(rows, cols):
    val = 0.03  # random.uniform(0.036, 0.107)
    # Full resolution
    noise_im1 = np.zeros((rows, cols))
    noise_im1 = random_noise(noise_im1, mode='gaussian', var=val ** 2, clip=False)

    # Half resolution
    noise_im2 = np.zeros((rows // 2, cols // 2))
    noise_im2 = random_noise(noise_im2, mode='gaussian', var=(val * 2) ** 2, clip=False)  # Use val*2 (needs tuning...)
    noise_im2 = resize(noise_im2, (rows, cols))  # Upscale to original image size

    # Quarter resolution
    noise_im3 = np.zeros((rows // 4, cols // 4))
    noise_im3 = random_noise(noise_im3, mode='gaussian', var=(val * 4) ** 2, clip=False)  # Use val*4 (needs tuning...)
    noise_im3 = resize(noise_im3, (rows, cols))  # What is the interpolation method?

    noise_im = noise_im1 + noise_im2  + noise_im3  # Sum the noise in multiple resolutions (the mean of noise_im is around zero).
    return noise_im


def noiseGenerator(im):
    im_arr = np.asarray(im)

    rows, cols, color_channels = im_arr.shape

    rgba_array = np.zeros((rows, cols, color_channels ), 'float64')
    for channel in range(color_channels):
        rgba_array[..., channel] += gen_noise_mask(rows, cols)
    noisy_img = im_arr / 255 + rgba_array  # Add noise_im to the input image.
    noisy_img = np.round((255 * noisy_img)).clip(0, 255).astype(np.uint8)
    return Image.fromarray(noisy_img)

In [ ]:
import os
path = ''
new_path=''


directory = os.fsencode(path)
for file in os.listdir(directory):
     filename = os.fsdecode(file)
     path_to_file = f'{path}{filename}'
     path_to_new_file = f'{new_path}{filename}'
     filename_lower = filename.lower()
     if filename_lower.endswith(".jpeg") or filename_lower.endswith(".png") or filename_lower.endswith(".jpg"): 
          grainy = noiseGenerator(Image.open(fp=path_to_file))
          grainy.save(fp=path_to_new_file)
          

